In [10]:
import torch
import torch.nn as nn
from torch.nn  import functional as F
import mmap
import random

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)
blockSize = 64
batchSize = 32
iters = 3000
lr = 3e-4
evali = 250
n_embed = 128
n_head = 4
n_layer= 4
dropout = 0.2

cuda


In [11]:
with open ('opensource/vocab.txt', 'r', encoding='UTF-8') as f:
    text = f.read()

chars=sorted(set(text))
print(chars) 

vocab = len(chars)

['\n', ' ', '#', '&', "'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '‘', '’', '“', '”', '•']


In [12]:
char_to_int = { ch:i  for i,ch in enumerate(chars)}
int_to_char = { i:ch for i,ch in enumerate(chars)}

encode = lambda s:[char_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_char[i] for i in l])


In [13]:
def get_random_chunk(split):
    filename = "opensource/output_train.txt" if split == 'train' else "opensource/output_val.txt"
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            file_size = len(mm)
            start_pos =random.randint(0, (file_size) - blockSize*batchSize)
            
            mm.seek(start_pos)
            block = mm.read(blockSize*batchSize-1)
            
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r','')
            
            data = torch.tensor(encode(decoded_block), dtype=torch.long)
            
    return data
def getBatch(split):
    data =  get_random_chunk(split)
    ix = torch.randint(len(data) - blockSize,(batchSize,) )
    x = torch.stack([data[i:i+blockSize] for i in ix])
    y = torch.stack([data[i+1:i+blockSize+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y


In [14]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(evali)
        for k in range(evali):
            X, Y = getBatch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [15]:
class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(blockSize, blockSize)))

        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)  
        q = self.query(x)
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x) 
        out = wei @ v 
        return out

class MultiHeadAttention(nn.Module):
     
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range (num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embed)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out
    
class FeedForward(nn.Module):
    
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear (n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)
        
        
class Block(nn.Module):
    
    def __init__(self, n_embed, n_head):
        super().__init__()
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)
        
    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x+y)
        return x


class GPT(nn.Module):
    
    def __init__(self, vocab):
        super().__init__()
        self.token = nn.Embedding(vocab, n_embed)
        self.position = nn.Embedding(blockSize, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head=n_head) for _ in range(n_layer)])
        
        self.ln_f = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab)
        
        self.apply(self._init_weights)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        
    def forward(self, index, targets=None):
        B, T = index.shape
        
        tok_emb = self.token(index)
        pos_emb = self.position(torch.arange(T, device=device))
        
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape 
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_tokens):
        
        for _ in range (max_tokens):
            logits, loss = self.forward(index)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
            
        return index
    
model = GPT(vocab)
m = model.to(device)
    
# context = torch.zeros((1,1), dtype=torch.long, device=device)
# generated = decode(m.generate(context, max_tokens=500)[0].tolist())
# print(generated)    
    

In [16]:
import math

optimize = torch.optim.AdamW(model.parameters(), lr=lr)

for iter in range(iters):
    if iter % evali == 0:
        losses = estimate_loss()
        
        texpo = losses['train']
        tacc = str(math.exp(-expo)*100) + '%'
        vexpo = losses['val']
        vacc = str(math.exp(-expo)*100) + '%'
        print(f"step: {iter}, tloss:{texpo}, tacc:{tacc} ,vloss:{vexpo} acc:{vacc}")
        
    xb, yb = getBatch('train')
    
    logits, loss = model.forward(xb, yb)
    optimize.zero_grad(set_to_none=True)
    loss.backward()
    optimize.step()
    
print(loss.item(), vacc)
        

step: 0, tloss:4.420050144195557, vloss:4.419230937957764 acc:1.2043490915755914%
step: 250, tloss:2.5258982181549072, vloss:2.524726390838623 acc:8.008022047147833%
step: 500, tloss:2.2866036891937256, vloss:2.2951831817626953 acc:10.074293730912236%
step: 750, tloss:2.056591510772705, vloss:2.056973457336426 acc:12.784029909836104%
step: 1000, tloss:1.892093300819397, vloss:1.8754854202270508 acc:15.328054330691%
step: 1250, tloss:1.755278468132019, vloss:1.753538727760315 acc:17.31600915407192%
step: 1500, tloss:1.6492573022842407, vloss:1.6406209468841553 acc:19.385962837723437%
step: 1750, tloss:1.5434379577636719, vloss:1.5611655712127686 acc:20.989128533080343%
step: 2000, tloss:1.4689116477966309, vloss:1.481917381286621 acc:22.720163831069044%
step: 2250, tloss:1.4084292650222778, vloss:1.4039020538330078 acc:24.563660421945556%
step: 2500, tloss:1.3461555242538452, vloss:1.3602524995803833 acc:25.65959783962228%
step: 2750, tloss:1.3092138767242432, vloss:1.3083282709121704 a